In [ ]:
from sympy.utilities.iterables import variations
from collections import Counter
import matplotlib.pyplot as plt

import lieb_liniger_state as lls
import rho_form_factor as rff
import numpy as np
import sys
from sum_rule import *
import utils

%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [ ]:
L = N = 5
Imax = 12

rstate = lls.lieb_liniger_state(1, L, N)
rstate.calculate_all()

In [ ]:
perms = set(sorted(tuple(sorted(x)) for x in variations(range(-Imax, Imax + 1), N)))

In [ ]:
k = Counter(sorted(sum(x) for x in perms))

In [ ]:
plt.bar([z for z in k], [k[z] for z in k])
plt.show()
print("# of states:", sum([k[z] for z in k]))

In [ ]:
dsf_data = {}
for i, a in enumerate(perms):
    lstate = lls.lieb_liniger_state(1, L, N, list(a))
    lstate.calculate_all()
    lstate.ff = rff.calculate_normalized_form_factor(lstate, rstate)
    if lstate.integer_momentum in dsf_data.keys():
        dsf_data[lstate.integer_momentum].append(lstate)
    else:
        dsf_data[lstate.integer_momentum] = [lstate]
    sys.stdout.write(f"{str(i)}\r")
    sys.stdout.flush()

In [ ]:
saturations = []
for i, states in sorted(dsf_data.items()):
    if i != 0:
        saturations.append(left_side(states, rstate.energy) / right_side(i, L, N))

In [ ]:
data = list(zip(*[[z for z in range(-max(dsf_data.keys()), max(dsf_data.keys()) + 1) if z != 0], saturations]))

largest_attained = Imax #max(dsf_data.keys())

plt.bar([z[0] for z in data if abs(z[0]) <= largest_attained], [z[1] for z in data if abs(z[0]) <= largest_attained])
plt.show()

print(sum([z[1] for z in data if abs(z[0]) <= largest_attained]) / len([z[1] for z in data if abs(z[0]) <= largest_attained]))

In [ ]:
no_ex_data = {}
for i, a in enumerate(perms):
    lstate = lls.lieb_liniger_state(1, L, N, list(a))
    lstate.calculate_all()
    no = utils.no_of_particle_hole_pairs(utils.map_to_entire_space(lstate.Is, 10), utils.map_to_entire_space(rstate.Is, Imax), N)
    lstate.ff = rff.calculate_normalized_form_factor(lstate, rstate)
    if no in no_ex_data.keys():
        no_ex_data[no].append(lstate)
    else:
        no_ex_data[no] = [lstate]

In [ ]:
form_facts = []
for k in sorted(no_ex_data.keys()):
    av = 0
    for state in no_ex_data[k]:
        av += np.abs(state.ff)
        if k == 0:
            print(state.Is, state.ff)
    av /= len(no_ex_data[k])
    form_facts.append(av)

In [ ]:
print(len(no_ex_data.keys()))
print(form_facts)
plt.bar(range(0, 17), form_facts)
plt.show()

In [ ]:
no_ex_data.keys()

In [ ]:
print(form_facts)

In [ ]:
plt.bar(range(0, 5), [len(no_ex_data[z]) for z in range(0, 5)])
plt.show()

In [ ]:
for t in no_ex_data[1]:
    print(t.Is, np.abs(t.ff))